In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
import glob

path = "../datasets/csv/" # use your path
all_movies = glob.glob(path + "/*.csv")

li = []

for filename in all_movies:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

all_movies_final = pd.concat(li, axis=0, ignore_index=True)
all_movies_final


,Unnamed: 0,rating,review date,review content
0,0,8.0,8 January 2021,Fern (Frances McDormand) is houseless and livi...
1,1,9.0,24 June 2021,When people put themselves in vulnerable posit...
2,2,6.0,2 August 2021,"""Nomadland"" is a new American movie that premi..."
3,3,7.0,18 February 2021,Nomadland is an exploration of people living i...
4,4,5.0,26 May 2021,"Frances McDormand loses her job in Empire, Nev..."
...,...,...,...,...
49069,504,10.0,5 November 2005,I first saw this movie in the theater when it ...
49070,505,10.0,11 January 2020,I saw this movie on the big screen when it was...
49071,506,10.0,22 June 2019,"Great movie!!!! I love the music, history and ..."
49072,507,10.0,22 November 2005,I am listening to the soundtrack as I pen thes...


1. Importing DataSets

In [9]:
data_all = pd.DataFrame(all_movies_final)
print(data_all)

data_all.columns = [c.replace(' ', '_') for c in data_all.columns]

data_all.dropna()

data_all_new = data_all[data_all["rating"] != -1]
print(pd.concat([data_all,data_all_new]).drop_duplicates(keep=False)["rating"] != -1)

       Unnamed:_0  ...                                     review_content
0               0  ...  Fern (Frances McDormand) is houseless and livi...
1               1  ...  When people put themselves in vulnerable posit...
2               2  ...  "Nomadland" is a new American movie that premi...
3               3  ...  Nomadland is an exploration of people living i...
4               4  ...  Frances McDormand loses her job in Empire, Nev...
...           ...  ...                                                ...
49069         504  ...  I first saw this movie in the theater when it ...
49070         505  ...  I saw this movie on the big screen when it was...
49071         506  ...  Great movie!!!! I love the music, history and ...
49072         507  ...  I am listening to the soundtrack as I pen thes...
49073         508  ...  The title infers a movie about dancing and a n...

[49074 rows x 4 columns]
28       False
34       False
66       False
67       False
80       False
         ..

In [ ]:
data_all_new['Sentiment'] = data_all_new['rating'].apply(lambda x: "positive" if x>=7 else("negative" if x<=4.9 else "neutral"))
data_all_new

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed:_0,rating,review_date,review_content,Sentiment
0,0,8.0,8 January 2021,Fern (Frances McDormand) is houseless and livi...,positive
1,1,9.0,24 June 2021,When people put themselves in vulnerable posit...,positive
2,2,6.0,2 August 2021,"""Nomadland"" is a new American movie that premi...",neutral
3,3,7.0,18 February 2021,Nomadland is an exploration of people living i...,positive
4,4,5.0,26 May 2021,"Frances McDormand loses her job in Empire, Nev...",neutral
...,...,...,...,...,...
49069,504,10.0,5 November 2005,I first saw this movie in the theater when it ...,positive
49070,505,10.0,11 January 2020,I saw this movie on the big screen when it was...,positive
49071,506,10.0,22 June 2019,"Great movie!!!! I love the music, history and ...",positive
49072,507,10.0,22 November 2005,I am listening to the soundtrack as I pen thes...,positive


In [ ]:
data_all_new1 = pd.DataFrame({"review_content1":data_all_new["review_content"], "Sentiment1":data_all_new["Sentiment"]})
data_all_new1

,review_content1,Sentiment1
0,Fern (Frances McDormand) is houseless and livi...,positive
1,When people put themselves in vulnerable posit...,positive
2,"""Nomadland"" is a new American movie that premi...",neutral
3,Nomadland is an exploration of people living i...,positive
4,"Frances McDormand loses her job in Empire, Nev...",neutral
...,...,...
49069,I first saw this movie in the theater when it ...,positive
49070,I saw this movie on the big screen when it was...,positive
49071,"Great movie!!!! I love the music, history and ...",positive
49072,I am listening to the soundtrack as I pen thes...,positive


In [ ]:
data_all_new1.shape

(44306, 2)

In [ ]:
data_all_new1['Sentiment1'].value_counts()

positive    33585
negative     6729
neutral      3992
Name: Sentiment1, dtype: int64

In [ ]:
data_all_new1.isnull().sum()

review_content1    0
Sentiment1         0
dtype: int64

In [ ]:
x = data_all_new1["review_content1"]
y = data_all_new1["Sentiment1"]

2. Data Cleaning

In [ ]:
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
def text_data_cleaning(sentence):
  doc = nlp(sentence)
  tokens = []
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)

  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens  

In [ ]:
text_data_cleaning("Hello everyone, it's a beautiful day outside!!!") #forexample

['hello', 'beautiful', 'day', 'outside']

Vectorization Feature Engineering (TF-IDF)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)

In [ ]:
classifier = LinearSVC()

3. Train the model

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((35444,), (8862,), (35444,), (8862,))

In [ ]:
x_train.head()

45909    Every once in a while, you watch a film so goo...
41887    This movie has to be one of the most moving fi...
42859    He cured his mental illness with his spirit.\n...
366      Movie 'Nomadland' is written and directed by C...
23853    Like most of us, when news comes from Hollywoo...
Name: review_content1, dtype: object

Fit the x_train and y_train

In [ ]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [ ]:
clf.fit(x_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7f31ca0e0dd0>)),
                ('clf', LinearSVC())])

4. Predict the Test Set Results

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
confusion_matrix(y_test, y_pred) #84.75% accuracy

array([[ 865,   94,  352],
       [ 230,  130,  427],
       [ 160,   88, 6516]])

In [ ]:
#classification report
print(classification_report(y_test, y_pred))
#we are getting almost 85% accuracy

              precision    recall  f1-score   support

    negative       0.69      0.66      0.67      1311
     neutral       0.42      0.17      0.24       787
    positive       0.89      0.96      0.93      6764

    accuracy                           0.85      8862
   macro avg       0.67      0.60      0.61      8862
weighted avg       0.82      0.85      0.83      8862



In [ ]:
accuracy_score(y_test, y_pred)

0.8475513428120063

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[ 865   94  352]
 [ 230  130  427]
 [ 160   88 6516]]
              precision    recall  f1-score   support

    negative       0.69      0.66      0.67      1311
     neutral       0.42      0.17      0.24       787
    positive       0.89      0.96      0.93      6764

    accuracy                           0.85      8862
   macro avg       0.67      0.60      0.61      8862
weighted avg       0.82      0.85      0.83      8862

0.8475513428120063


In [ ]:
clf.predict(["this movie was okay"])


array(['neutral'], dtype=object)

In [ ]:
clf.predict(["I hate this movie. It was a waste of my time"])


array(['negative'], dtype=object)

In [ ]:
clf.predict(["I love this movie"])

array(['positive'], dtype=object)